In [1]:
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import\
    DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.graph_traversal import __
import json


# TinkerPop 3.5.0 
## Tornado replaced 

Since release, the Python client has used Tornado for its websocket and other transport needs. While this generally worked well, in some cases, such as when working within Jupyter notebooks, the Gremlin client encountered conflicts when trying to create its event loop. Starting with the 3.5.0 release, the Python client no longer uses Tornado. Instead AIO HTTP is used. For most users this will have zero impact to their code. However any code using features provided by Tornado may have to be slightly revised to use AIO HTTP instead. The move to AIO HTTP removes the event loop issues experienced by users of the prior clients and opens the door for additional future improvements.

## max_content_length 

The Gremlin Java clients provide a setting that can be used to increase, or decrease, the maximum size of the result that can be returned to a client. The Tornado default is 10,240 bytes ( 10 * 10 * 1024) but the prior Python clients did not expose a way to set this value. Starting with the 3.5.0 Python client, transport specific arguments can now be provided when a remote connection is created. These are passed as kwargs to allow for easy expansion in the future without having to add additional arguments to the API each time a new setting is needed. If a value is not provided for max_content_length the same default 10,0240 byte value is used. The code below shows an example of how this works.


In [2]:
server = '172.17.0.2'
port = 8182
endpoint = 'ws://' + server + ':' + str(port) + '/gremlin'
transport_args = {'max_content_length': 20000}
connection = DriverRemoteConnection(endpoint, 'g', **transport_args)
g = traversal().withRemote(connection)
# results = g.V('1','2','3').valueMap().toList()
# print(results)

In [3]:
g.addV('person').property('name', 'serg').as_('m'). \
        addV('person').property('name', 'alex').as_('v'). \
        addE('knows').from_('m').to('v').iterate()

[['addV', 'person'], ['property', 'name', 'serg'], ['as', 'm'], ['addV', 'person'], ['property', 'name', 'alex'], ['as', 'v'], ['addE', 'knows'], ['from', 'm'], ['to', 'v'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [4]:
    print(
        "marko: " + json.dumps(g.V().has('person', 'name', 'serg')
                              .valueMap().
                              next())
    )

marko: {"name": ["serg"]}


In [5]:
print(
        "who marko knows: " + json.dumps(g.V().has('person', 'name', 'serg')
                                        .out('knows')
                                        .valueMap()
                                        .next())
    )

who marko knows: {"name": ["alex"]}


In [6]:
g.V().has('person', 'name', 'serg').out('knows').valueMap().toList()

[{'name': ['alex']}]

## Load a dataset

In [7]:
g.V().drop().iterate()
g.io('/home/serg/projects/Bigdata/dataset/air-routes.graph.xml').read().iterate()

[['io', '/home/serg/projects/Bigdata/dataset/air-routes.graph.xml'], ['read'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

## has(), hasLabel(), hasId()

In [8]:
findId = g.V().hasLabel('airport').has('code', 'SYD').next()
g.V().hasLabel('airport').has('code', 'SYD').next() == g.V().has('airport', 'code', 'SYD').next() == g.V().hasId(findId).next()

True

## Count

In [9]:
print(g.V().count().next())
print(g.V().hasLabel('country').count().next())
print(g.V().hasLabel('airport').count().next())

3619
237
3374


## valueMap()


In [10]:
g.V().has('airport', 'code', 'SYD').valueMap().next()

{'code': ['SYD'],
 'type': ['airport'],
 'desc': ['Sydney Kingsford Smith'],
 'country': ['AU'],
 'longest': [12999],
 'city': ['Sydney'],
 'elev': [21],
 'icao': ['YSSY'],
 'lon': [151.177001953125],
 'region': ['AU-NSW'],
 'runways': [3],
 'lat': [-33.9460983276367]}

In [11]:
# Check all data in request using next()
results = g.V(findId).values()
count = g.V(findId).values().count().next()
for c in range(count):
  print(results.next())

SYD
airport
Sydney Kingsford Smith
AU
12999
Sydney
21
YSSY
151.177001953125
AU-NSW
3
-33.9460983276367


In [12]:
req = g.V(findId).values('desc', 'runways')
print(f'{req.next()} airport has {req.next()} runways...')


Sydney Kingsford Smith airport has 3 runways...


## unfold() fold()

In [13]:

req = g.V(findId).valueMap().unfold()
count = g.V(findId).values().count().next()
for i in range(count):
  print(f'{str(i)}, {req.next()}')


0, {'code': ['SYD']}
1, {'type': ['airport']}
2, {'desc': ['Sydney Kingsford Smith']}
3, {'country': ['AU']}
4, {'longest': [12999]}
5, {'city': ['Sydney']}
6, {'elev': [21]}
7, {'icao': ['YSSY']}
8, {'lon': [151.177001953125]}
9, {'region': ['AU-NSW']}
10, {'runways': [3]}
11, {'lat': [-33.9460983276367]}


In [14]:
req = g.V(findId).valueMap().unfold().fold().next()
req

[{'code': ['SYD']},
 {'type': ['airport']},
 {'desc': ['Sydney Kingsford Smith']},
 {'country': ['AU']},
 {'longest': [12999]},
 {'city': ['Sydney']},
 {'elev': [21]},
 {'icao': ['YSSY']},
 {'lon': [151.177001953125]},
 {'region': ['AU-NSW']},
 {'runways': [3]},
 {'lat': [-33.9460983276367]}]

## hasNot(), not()

In [16]:
req = g.V().hasNot('region').valueMap()
count = g.V().hasNot('region').count().next()
for c in range(count):
    if c == 10:
        break
    print(req.next())

{'code': ['AL'], 'type': ['country'], 'desc': ['Albania']}
{'code': ['BB'], 'type': ['country'], 'desc': ['Barbados']}
{'code': ['BO'], 'type': ['country'], 'desc': ['Bolivia']}
{'code': ['BQ'], 'type': ['country'], 'desc': ['Bonaire - Sint Eustatius and Saba']}
{'code': ['BG'], 'type': ['country'], 'desc': ['Bulgaria']}
{'code': ['BI'], 'type': ['country'], 'desc': ['Burundi']}
{'code': ['KY'], 'type': ['country'], 'desc': ['Cayman Islands']}
{'code': ['CF'], 'type': ['country'], 'desc': ['Central African Republic']}
{'code': ['CC'], 'type': ['country'], 'desc': ['Cocos (Keeling) Islands']}
{'code': ['CD'], 'type': ['country'], 'desc': ['Democratic Republic of the Congo']}


In [17]:
same_count = g.V().not_(__.has('region')).count().next()

In [18]:
g.V().hasLabel('airport').next(3)

[v[5849112], v[5853208], v[5857304]]

In [19]:
g.E().hasLabel('route').count().next()

43400

In [20]:
results = g.V(findId).inE('route').outV().values('desc')
count = g.V(findId).inE('route').outV().count().next()
for c in range(count):
    print(results.next())

Melbourne International Airport
Wellington International Airport
Manila, Ninoy Aquino International Airport
Santiago, Comodoro Arturo Merino Benitez International Airport
Cairns International Airport
Darwin International Airport
Indira Gandhi International Airport
Auckland International Airport
Christchurch International Airport
Tokyo Haneda International Airport
Doha, Hamad International Airport
Alice Springs Airport
Dallas/Fort Worth International Airport
Soekarno-Hatta International Airport
Honolulu International Airport
Vancouver
Guangzhou Baiyun International Airport
Perth International Airport
Canberra International Airport
Seoul, Incheon International Airport
Sunshine Coast Airport
Dubai International Airport
Hong Kong - Chek Lap Kok International Airport
San Francisco International Airport
Singapore, Changi International Airport
Kuala Lumpur International Airport
Ho Chi Minh City, Tan Son Nhat International Airport
Ayers Rock Connellan Airport
Gold Coast Airport
Abu Dhabi Inter

## Get types of vertices

In [21]:
a = g.V().groupCount().by(T.label).next()
print(a)

{'continent': 7, 'country': 237, 'version': 1, 'airport': 3374}


In [22]:
g.V().hasLabel('airport').groupCount().by('country').select('FR','GR','BE').next()

{'FR': 58, 'GR': 39, 'BE': 5}

In [23]:
g.V().hasLabel('continent').group().\
    by('code').by(__.out().count()).next()

{'EU': 583, 'AS': 932, 'NA': 978, 'OC': 284, 'AF': 294, 'AN': 0, 'SA': 303}

In [24]:
g.V().hasLabel('airport').limit(5).group().\
    by('code').by('runways').next()

{'ORD': [8], 'BNA': [4], 'BWI': [3], 'IAD': [4], 'FLL': [2]}